loading libraries

In [ ]:
import numpy as np
import json as js
from keras import models
import cv2
from os import listdir as ld
from os import path
from tqdm import tqdm

Loading paths

In [7]:
with open("config_CNN.json",'r') as file:
    paths = js.load(file)

loading model

In [8]:
model = models.load_model(paths["Trained_model"])

Evaluation on validation data

In [22]:
def compute_iou(box1, box2):
    # Calculate intersection area
    intersection_width = min(box1.right, box2.right) - max(box1.left, box2.left)
    intersection_height = min(box1.bottom, box2.bottom) - max(box1.top, box2.top)
    
    if intersection_width <= 0 or intersection_height <= 0:
        return 0
    
    intersection_area = intersection_width * intersection_height

    # Calculate union area
    box1_area = (box1.right - box1.left) * (box1.bottom - box1.top)
    box2_area = (box2.right - box2.left) * (box2.bottom - box2.top)
    
    union_area = box1_area + box2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area
    return iou

TP, FP, FN = 0, 0, 0

In [ ]:
val_data_path = paths["Validation_resized"]

with open(paths["Preprocessed_Validation_json"],'r') as file:
    val_json = js.load(file)

for fname in tqdm(ld(val_data_path)):
    if fname.endswith((".jpg", ".png", ".jpeg")):
        img_path = path.join(val_data_path, fname)
        txt_path = img_path.replace(".jpg", ".txt").replace(".png", ".txt")  
        img = cv2.imread(img_path)
        img_input = np.expand_dims(img, axis=0)  
        preds = model.predict(img_input, verbose = 0)  
        
        if path.exists(txt_path):
            with open(txt_path, "r") as f:
                gt_boxes = [list(map(float, line.strip().split()[1:])) for line in f]
        else:
            gt_boxes = []

        matched = set()

        for pred_box in preds:
            iou_scores = [compute_iou(pred_box[1:], gt) for gt in gt_boxes]
            max_iou = max(iou_scores) if iou_scores else 0

            if max_iou > 0.5 and iou_scores.index(max_iou) not in matched:
                TP += 1
                matched.add(iou_scores.index(max_iou))
                
            else:
                FP += 1

        FN += len(gt_boxes) - len(matched)

100%|██████████| 5000/5000 [03:50<00:00, 21.68it/s]


In [24]:
# Compute metrics
precision = TP / (TP + FP + 1e-6)
recall = TP / (TP + FN + 1e-6)
f1_score = 2 * precision * recall / (precision + recall + 1e-6)

print(f"\nPrecision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1_score:.4f}")


Precision: 0.0000
Recall:    0.0000
F1 Score:  0.0000
